In [561]:
using JuMP
using CPLEX
using Distributions
using LinearAlgebra
using Statistics
using Dates
using DataFrames
using SDDP
using Plots
import CSV
using JSON
try
    using Revise
catch e
    @warn "Error initializing Revise" exception=(e, catch_backtrace())
end

includet(pwd() * "\\Water_Regulation\\WaterRegulation.jl")
using .WaterRegulation

In [562]:
filepath_Ljungan = pwd() * "\\Water_Regulation\\TestDataWaterRegulation\\Ljungan.json"
filepath_prices = pwd() *  "\\Data\\Spot Prices\\prices_df.csv"
filepath_results = pwd() * "\\Results\\LambdaZero\\"
R, K, J = read_data(filepath_Ljungan)
print() 

j = J[1]
O, K_O = OtherParticipant(J, j , R)
K_j = [j.plants[1]]
K_O = [O.plants[1]]
pj = j.participationrate
pO = O.participationrate

println("Participation rate $(j.name): \n " , pj)
println("Individual Reservoir of $(j.name): \n", j.individualreservoir)
println("Participation rate $(O.name): \n ", pO)
println("K_O : $(K_O) and spillage $(K_O[1].spillreference),\nK_j : $(K_j) and spillage $(K_j[1].spillreference)")

Stages = 2 # Days in Time Horizon <-> Stages
T = 24
scenario_count = 1

Prices = [floor.(rand(T), sigdigits=3) for i in 1:scenario_count]
Inflows = [10.0]

# Balancing Penalties
mu_up = 1.0
mu_down = 0.2

Omega = [(price = p, inflow = v) for p in Prices for v in Inflows]
P = [1/length(Omega) for om in Omega]

Participation rate Sydkraft: 
 ________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

Individual Reservoir of Sydkraft: 
________________________________
Flasjon  | 15000.0 
Holsmjon | 18000.0 

Participation rate Other: 
 ________________________________
Flasjon  | 2.68    
Holsmjon | 2.68    

K_O : HydropowerPlant[Parteboda] and spillage 1.4,
K_j : HydropowerPlant[Flasjo] and spillage 0.58

1-element Vector{Float64}:
 1.0

### Short-Term Scheduling of Hydropower Plants

After Day-Ahead Market Bidding and Adjusted Flow Calculation, we have the opportunity to renominate.  
It does not matter wether the agent is anticipatory or nonanticipatory: In any case the players have to abide to the water regulation rules.  

The optimizers' goal is to disperse water correctly and solve the unit commitment problem of which power plants to use. Additionally, we minimize the amount of balancing we haave to do.  

Bidding is removed from the optimization model. We now only have to work with the known constraint of demand in the first stage:
$$
y_t = \sum\limits_{k \in \mathcal{K}} w_{k,t} + \sum\limits_{r \in \mathcal{R}} P^\text{swap}_r + z^+_t - z^-_t 
$$

The subsequent stages don't have a known demand, as bidding is only done for the first stage. In the next stages we leave out the demand constraint.


The goal is to find an optimal nomination and production schedul. The nomination includes information of how we value the reservoirs' contents.   
In the objective we can either leave out or keep the revenues from day-ahead obligations. As they have to be fulfilled anyways it does not have any impact on the optimal solution.

##### Parameters
* Reference Flow by Water Regulation Company
* Day-Ahead Obligations for every hour
* Adjusted Flow -> Other's nomination

Power Swap and own nomination can also be extracted but are irrelevant as they will be redetermined.

In [563]:
Qref = Dict{Reservoir, Float64}(r => 0.8 for r in R)
QnomO = Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}((participant = j, reservoir = r) => 0.5 for r in R for j in J)
println(QnomO)
y::Vector{Float64} = 2 .* rand(24)
S = 0.2
println("Obligation: ", y)
println(j.participationrate)
for r in R
    println(min([k.spillreference for k in filter(k -> k.reservoir == r,K)]...))
end

Dict{NamedTuple{(:participant, :reservoir), Tuple{Participant, Reservoir}}, Float64}((participant = Sydkraft, reservoir = Holsmjon) => 0.5, (participant = Fortum, reservoir = Flasjon) => 0.5, (participant = Statkraft, reservoir = Flasjon) => 0.5, (participant = Fortum, reservoir = Holsmjon) => 0.5, (participant = Statkraft, reservoir = Holsmjon) => 0.5, (participant = Sydkraft, reservoir = Flasjon) => 0.5)
Obligation: [0.9185366584385861, 0.8960662151397083, 1.7182091180089012, 1.8373170889287374, 0.6315464157261552, 1.8746159380334897, 0.39699559467006584, 0.9942022407205848, 0.7437609411276231, 0.37425151469900264, 0.8346893894576382, 1.3286580622585797, 0.7883768028343812, 1.0385976225746294, 0.10457260505896748, 1.5256194827120946, 0.2642608858566886, 1.8790414817478847, 0.8555756106688477, 1.2577763041472063, 0.24471654554170175, 1.009407464015287, 0.4197610609197058, 0.4457132567187041]
________________________________
Flasjon  | 1.84    
Holsmjon | 0.0     

0.58
1.4


In [585]:
function subproblem_builder_short(subproblem::Model, node::Int64)
    # State Variables
    @variable(subproblem, 0 <= l[r = R] <= r.maxvolume, SDDP.State, initial_value = r.currentvolume)
    @variable(subproblem, lind[r = R], SDDP.State, initial_value = j.individualreservoir[r])
    @variable(subproblem, u_start[k = K_j], SDDP.State, initial_value = 1, Bin)
    # Control Variables
    @variable(subproblem, 0 <= Qnom[r = R] <= max([k.spillreference for k in filter(k -> k.reservoir in find_ds_reservoirs(r), K)]...))
    @variable(subproblem, Qadj[r = R] >= 0)
    @variable(subproblem, d[t = 1:T, k = K_j], Bin)
    @variable(subproblem, u[t = 1:T, k = K_j], Bin)
    @variable(subproblem, BALANCE_INDICATOR[r = R], Bin)
    @variable(subproblem, 0 <= w[t = 1:T, k = K_j] <= k.equivalent * k.spillreference)
    @variable(subproblem, 0 <= Qeff[t = 1:T, k = K_j] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t = 1:T, r = R])
    @variable(subproblem, s[r = R] >= 0)
    @variable(subproblem, a[r = R])
    # Random Variables
    @variable(subproblem, f[r = R] >= 0)
    # Transition Function
    if node == 1
        @constraint(subproblem, balance[r = R], l[r].out == l[r].in - T * Qadj[r] - s[r])
    else
        @constraint(subproblem, balance[r = R], l[r].out == l[r].in - T * Qnom[r] + f[r] * T - s[r])
    end
    @constraint(subproblem, balance_ind[r = R], lind[r].out == lind[r].in - T * (Qnom[r] - Qref[r])- s[r]) 
    @constraint(subproblem, startcond[k = K_j], u_start[k].in == u[1,k])
    @constraint(subproblem, endcond[k = K_j], u_start[k].out == u[T,k])
    # Constraints
    if node == 1
        @variable(subproblem, z_up[t = 1:T] >= 0)
        @variable(subproblem, z_down[t = 1:T] >= 0)
        @variable(subproblem, Pswap[r = R])
        @variable(subproblem, Pover[k = K_O] >= 0)
        @constraint(subproblem, obligation[t = 1:T], y[t]  == sum(w[t,k] for k in K_j) + sum(Pswap[r] for r in R) + z_up[t] - z_down[t])
        @constraint(subproblem, powerswap[r = R], Pswap[r] == j.participationrate[r] * (Qnom[r] - Qadj[r]) - sum(Pover[k] for k in K_O))
        @constraint(subproblem, overnomination[k = K_O], Pover[k] >= k.equivalent * (Qadj[k.reservoir] - k.spillreference))
        @constraint(subproblem, adjustedflow[r = R], Qadj[r] == (Qnom[r] * j.participationrate[r] + QnomO[(participant = j, reservoir = r)] * O.participationrate[r]) / (j.participationrate[r] + O.participationrate[r]))
        @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qadj[r])
    else
        @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qnom[r])
    end
    @constraint(subproblem, nbal1[r = R], BALANCE_INDICATOR[r] => {Qnom[r] <= Qref[r]})
    @constraint(subproblem, nbal2[r = R], !BALANCE_INDICATOR[r] => {0 <= lind[r].in})
    @constraint(subproblem, NoSpill[r = R], BALANCE_INDICATOR[r] => {sum(Qnom[r_up] for r_up in find_us_reservoir(r)) <= min([k.spillreference for k in filter(k -> k.reservoir == r, K)]...)})
    @constraint(subproblem, active[t = 1:T, k = K_j], w[t,k] <= u[t,k] * k.spillreference * k.equivalent)
    @constraint(subproblem, startup[t = 1:T-1, k = K_j], d[t,k] >= u[t+1,k] - u[t,k])
    @constraint(subproblem, production[t = 1:T, k = K_j], w[t,k] <= Qeff[t,k] * k.equivalent)
    @constraint(subproblem, realwater[t = 1:T, k = K_j], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
    @constraint(subproblem, watervalue[r = R], a[r] >= j.participationrate[r] * (lind[r].in - lind[r].out) * 0.5)
    if node > 1
        SDDP.parameterize(subproblem, Omega, P) do om
            # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
            # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
            # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
            for r in R
                JuMP.fix(f[r], om.inflow, force=true)
            end
            # Include only active variables in stageobjective
            if node == 1
                # Fixed Price and Obligation
                @stageobjective(subproblem, sum(price[t] * y[t] - mu_up * z_up[t] + mu_down * z_down[t] - S * sum(d[t,k] for k in K_j) for t in 1:T) - sum(a[r] for r in R))
            else
                @stageobjective(subproblem, sum(om.price[t] * sum(w[t,k] for k in K_j) - S * sum(d[t,k] for k in K_j) for t in 1:T) - sum(a[r] for r in R))
            end
        end
    end
    return
end

subproblem_builder_short (generic function with 1 method)

In [586]:
model_short = SDDP.LinearPolicyGraph(
    subproblem_builder_short,
    stages = Stages,
    sense = :Max,
    upper_bound = sum(sum(sum(k.spillreference * k.equivalent for k in K_j) for t in 1:T) for stage in 1:Stages),
    optimizer = CPLEX.Optimizer
)

A policy graph with 2 nodes.
 Node indices: 1, 2


In [587]:
SDDP.train(model_short; iteration_limit=10)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-23
-------------------------------------------------------------------
problem
  nodes           : 2
  state variables : 5
  scenarios       : 1.00000e+00
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                                                                   : [167, 218]
  AffExpr in MOI.EqualTo{Float64}                                               : [8, 36]
  AffExpr in MOI.GreaterThan{Float64}                                           : [25, 26]
  AffExpr in MOI.LessThan{Float64}                                              : [72, 72]
  VariableRef in MOI.GreaterThan{Float64}                                       : [107, 155]
  VariableRef in MOI.LessThan{Float64}                                          : [53, 53]
 


         1   1.766400e+01  8.630400e+00  2.199984e-02         4   1


        10   1.766400e+01  8.630400e+00  1.889999e-01        42   1
-------------------------------------------------------------------
status         : iteration_limit
total time (s) : 1.889999e-01
total solves   : 42
best bound     :  8.630400e+00
simulation ci  :  1.766400e+01 ± 0.000000e+00
numeric issues : 0
-------------------------------------------------------------------



In [588]:
rule_short = SDDP.DecisionRule(model_short; node = 1)
sol_short = SDDP.evaluate(
    rule_short;
    incoming_state = Dict(Symbol("l[$(r.dischargepoint)]") => r.currentvolume for r in R),
    controls_to_record = [:Qnom, :Qreal, :w, :u, :Pswap, :Pover, :z_down, :z_up],
)
println(sol_short.controls[:Qnom])
println(sol_short.controls[:z_down])
println(sol_short.controls[:z_up])
println(sol_short.controls[:w])
println(sol_short.controls[:Pswap])
println(y)
println(Prices[1])

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Reservoir[Flasjon, Holsmjon]
And data, a 2-element Vector{Float64}:
 2.6500000000000004
 0.0
[1.427056261915397, 1.4495267052142748, 0.627383802345082, 0.5082758314252458, 1.714046504627828, 0.47097698232049345, 1.9485973256839173, 1.3513906796333983, 1.60183197922636, 1.9713414056549805, 1.510903530896345, 1.0169348580954034, 1.557216117519602, 1.3069952977793537, 2.2410203152950157, 0.8199734376418886, 2.0813320344972945, 0.4665514386060985, 1.4900173096851355, 1.0878166162067768, 2.100876374812281, 1.3361854563386961, 1.9258318594342774, 1.899879663635279]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(24)
    Dimension 2, HydropowerPlant[Flasjo]


And data, a 24×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, Reservoir[Flasjon, Holsmjon]
And data, a 2-element Vector{Float64}:
 2.345592920353983
 0.0
[0.9185366584385861, 0.8960662151397083, 1.7182091180089012, 1.8373170889287374, 0.6315464157261552, 1.8746159380334897, 0.39699559467006584, 0.9942022407205848, 0.7437609411276231, 0.37425151469900264, 0.8346893894576382, 1.3286580622585797, 0.7883768028343812, 1.0385976225746294, 0.10457260505896748, 1.5256194827120946, 0.2642608858566886, 1.8790414817478847, 0.8555756106688477, 1.2577763041472063, 0.24471654554170175, 1.009407464015287, 0.4197610609197058, 0.4457132567187041]
[0.266, 0.278, 0.567, 0.943, 0.911, 0.88, 0.743, 0.824, 0.282, 0.0437, 0.481, 0.294, 0.322, 0.181, 0.641, 0.941, 0.863, 0.969, 0.992, 0.675, 0.207, 0.957, 0.219, 0.738]


### Real Time Balancing and Water Scheduling

Even after going through two optimization problems, we are still not done yet.  
After all participants communicated their renomination, a new adjusted flow has been calculated.  
Although multiple renominations are theoretically possible, we choose to only have one round of nomination before and after market clearing each. That is because no new information outside of the water regulation procedure is gained.  

The last problem is simply a problem of real time balancing and unit commitment, as the amount of water to be discharged is determined. The Water Value function doesn't play a role anymore. We minimize costs:

$$
\min \; \rightarrow \; \sum\limits_{t \in \mathcal{T}} \left( \sum\limits_{k \in \mathcal{K}} ( S_k \cdot \delta_{k,t} ) +  \mu^+ z^+_t - \mu^- z^-_t  \right)
$$

As a Parameter we get:

* Adjusted Flow
* Power Swap
* Obligation of Day Ahead Market

We Decide:

* Flow of water at every hour
* Balancing Decisions (sell surplus and purchase necessary amount to sell)
* Startups of each power plant

In [589]:
Qadj = Dict{Reservoir, Float64}(r => 0.8 for r in R)
Pswap = Dict{Reservoir, Float64}(r => 0.0 for r in R)

Dict{Reservoir, Float64} with 2 entries:
  Flasjon  => 0.0
  Holsmjon => 0.0

In [590]:
function subproblem_builder_balancing(subproblem::Model, node::Int64)
    # State Variables
    @variable(subproblem, u_start[k = K_j], SDDP.State, initial_value = 0, Bin)
    # Control Variables
    @variable(subproblem, d[t = 1:T, k = K_j], Bin)
    @variable(subproblem, u[t = 1:T, k = K_j], Bin)
    @variable(subproblem, 0 <= w[t = 1:T, k = K_j] <= k.equivalent * k.spillreference)
    @variable(subproblem, z_up[t = 1:T] >= 0)
    @variable(subproblem, z_down[t = 1:T] >= 0)
    @variable(subproblem, 0 <= Qeff[t = 1:T, k = K_j] <= k.spillreference)
    @variable(subproblem, 0 <= Qreal[t = 1:T, r = R])
    # Transition Function
    @constraint(subproblem, startcond[k = K_j], u_start[k].in == u[1,k])
    @constraint(subproblem, endcond[k = K_j], u_start[k].out == u[T,k])
    # Constraints
    @constraint(subproblem, nomination[r = R], sum(Qreal[t,r] for t in 1:T) == T * Qadj[r])
    @constraint(subproblem, obligation[t = 1:T], y[t]  == sum(w[t,k] for k in K_j) + sum(Pswap[r] for r in R) + z_up[t] - z_down[t])
    @constraint(subproblem, active[t = 1:T, k = K_j], w[t,k] <= u[t,k] * k.spillreference * k.equivalent)
    @constraint(subproblem, startup[t = 1:T-1, k = K_j], d[t,k] >= u[t+1,k] - u[t,k])
    @constraint(subproblem, production[t = 1:T, k = K_j], w[t,k] <= Qeff[t,k] * k.equivalent)
    @constraint(subproblem, realwater[t = 1:T, k = K_j], Qeff[t,k] <= sum(Qreal[t,r] for r in find_us_reservoir(k.reservoir)))
    SDDP.parameterize(subproblem, Omega, P) do om
        # We have to make sure that depending on the market clearing price, the coefficients are set accordingly.
        # The recourse action only applies to the real delivery, determined by the uncertain price. The other restricitions become inactive, else they make the problem infeasible.
        # The constraints that are relevant are maiintained in Scenario_Index for every current time step.
        for r in R
            JuMP.fix(f[r], om.inflow, force=true)
            JuMP.set_normalized_rhs(adjustedflow[r], pO[r] * om.nomination)
        end
        # Include only active variables in stageobjective
        @stageobjective(subproblem, sum(mu_up * z_up[t] - mu_down * z_down[t]  + S * sum(d[t,k] for k in K_j) for t in 1:T))
    end
    return
end

subproblem_builder_balancing (generic function with 1 method)

In [591]:
model_balancing = SDDP.LinearPolicyGraph(
    subproblem_builder_balancing,
    stages = 1,
    sense = :Min,
    lower_bound = sum(sum(y[t] * mu_down for t in 1:T) for s in 1:Stages),
    optimizer = CPLEX.Optimizer
)

A policy graph with 1 nodes.
 Node indices: 1


As the problem is entirely deterministic (mixed-binary), we can  use a JuMP model to solve it entirely

In [592]:
function ShortTermBalancing(
        R::Vector{Reservoir},
        j::Participant,
        Qadj::Dict{Reservoir, Float64},
        Pswap::Dict{Reservoir, Float64},
        y::Vector{Float64};
        S = 0.2,
        T = 24,
        mu_up = 1.0,
        mu_down = 0.01)
    K_j = j.plants
    model_balancing = JuMP.Model(CPLEX.Optimizer)
    # Variables
    @variable(model_balancing, 0 <= z_up[t = 1:T])
    @variable(model_balancing, 0 <= z_down[t = 1:T])
    @variable(model_balancing, 0 <= w[t = 1:T, k = K_j] <= k.equivalent * k.spillreference)
    @variable(model_balancing, u[t = 1:T, k = K_j], Bin)
    @variable(model_balancing, d[t = 1:T, k = K_j], Bin)
    @variable(model_balancing, Qreal[t = 1:T, r = R])

    # Constraints
    @constraint(model_balancing, obligation[t = 1:T], y[t] == sum(w[t,k] for k in K_j) + sum(Pswap[r] for r in R) + z_up[t] - z_down[t])
    @constraint(model_balancing, adjustedflow[r = R], sum(Qreal[t, r] for t in 1:T) == T * Qadj[r])
    @constraint(model_balancing, production[t = 1:T, k = K_j], w[t,k] <= sum(Qreal[t, r] for r in find_us_reservoir(k.reservoir)) * k.equivalent)
    @constraint(model_balancing, unit[t = 1:T, k = K_j], w[t,k] <= u[t,k] * k.equivalent * k.spillreference)
    @constraint(model_balancing, startup[t = 2:T, k = K_j], d[t ,k] >= u[t,k] - u[t-1,k])
    # Objective
    @objective(model_balancing, Min, sum(mu_up * z_up[t] - mu_down * z_down[t] + sum(S * d[t, k] for k in K_j) for t in 1:T))
    optimize!(model_balancing)
    println(objective_value(model_balancing))
    return model_balancing, value.(z_up), value.(z_down)
end

ShortTermBalancing (generic function with 3 methods)

In [593]:
mb, z_up, z_down = ShortTermBalancing(R, J[2], Qadj, Pswap, y)

println("Upregulation: ", z_up)
println("Downregulation: ", z_down)


Root node processing (before b&c):
  Real time             =    0.00 sec. (1.30 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (1.30 ticks)
CPLEX Error  1217: No solution exists.
CPLEX Error  1217: No solution exists.
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Found incumbent of value 22.382268 after 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 155 rows and 158 columns.
Reduced MIP has 84 rows, 147 columns, and 328 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.22 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 11 rows and 39 columns.
Reduced MIP has 73 rows, 108 columns, and 214 nonzeros.
Reduced MIP has 0 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time 

In [594]:
solution_summary(mb; verbose = true)

* Solver : CPLEX

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "integer optimal solution"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 3.13836e+00
  Objective bound    : 3.13836e+00
  Relative gap       : 0.00000e+00
  Dual objective value : 3.13836e+00
  Primal solution :
    Qreal[1,Flasjon] : 7.25000e-01
    Qreal[1,Holsmjon] : 7.25000e-01
    Qreal[10,Flasjon] : 7.25000e-01
    Qreal[10,Holsmjon] : 7.25000e-01
    Qreal[11,Flasjon] : 7.25000e-01
    Qreal[11,Holsmjon] : 7.25000e-01
    Qreal[12,Flasjon] : 1.02500e+00
    Qreal[12,Holsmjon] : 1.02500e+00
    Qreal[13,Flasjon] : 7.25000e-01
    Qreal[13,Holsmjon] : 7.25000e-01
    Qreal[14,Flasjon] : 7.25000e-01
    Qreal[14,Holsmjon] : 7.25000e-01
    Qreal[15,Flasjon] : 7.25000e-01
    Qreal[15,Holsmjon] : 7.25000e-01
    Qreal[16,Flasjon] : 7.25000e-01
    Qreal[16,Holsmjon] : 7.25000e-01
    Qreal[